# Grid notation of permutations

## Libraries

In [ ]:
import random
import sympy
sympy.init_printing()

# import plotly.graph_objects as go

# ----- Debugger -----
# from IPython.core.debugger import Pdb; Pdb().set_trace()
# --------------------

In [ ]:
import sys
sys.path.append("../modules")

import Permutations as pm
from Grid import *

## Sequence of reduction

In [ ]:
N = 16
img = list(range(N)); random.shuffle(img)

prm = pm.Permutation(img)
display(sympy.Matrix(prm.two_line()))  # , prm.image, prm.size)
cd = pm.cycle_decomp(prm); print("{}, {}".format(cd, len(cd)))

G = Grid(prm)
G.show()

In [ ]:
grid = G.copy(); G.show()#; print("\n↓ \n")
nums = []

while grid.size > 1:
    bln = False if random.randint(0,1) == 0 else True
    new = grid.transform(bln)
    if type(new) == dict:
        nums += [new["num"]]
        grid = new["grid"]
    else:
        grid = new
    #---
    print("\n{} ↓   nums = {}\n".format("BR" if bln else "TL",nums))
    grid.show()
else:
    nums += [grid.matrix[0][0]]
    print("numbers:{}, total:{}".format(nums, sum(nums)))